## Movie Review

In [1]:
from load_sentiment import load_sentiment_data
from text_utils import build_dict, generate_sequence
from models import build_lstm, build_cnn, build_cnn_lstm, build_bidirectional_lstm

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN 5103)


In [2]:
dirpath = "/home/ec2-user/data/rt-polaritydata/"
max_features = 10000

X_train, X_test, Y_train, Y_test, worddict = load_sentiment_data(dirpath, max_tokens=max_features)

Building dictionary.. 201209  total words  20247  unique words
Filtering to 10000  unique words


In [3]:
nb_classes = len(set(Y_train + Y_test))
print('Num. classes:', nb_classes)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

maxlen_train = max([len(x) for x in X_train])
maxlen_test  = max([len(x) for x in X_test])
print('Max length train', maxlen_train)
print('Max length tet', maxlen_test)

('Num. classes:', 2)
(9595, 'train sequences')
(1067, 'test sequences')
('Max length train', 56)
('Max length tet', 59)


In [4]:
maxlen = 60  
batch_size = 32

In [5]:
from keras.preprocessing import sequence

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
('X_train shape:', (9595, 60))
('X_test shape:', (1067, 60))


In [6]:
from keras.utils import np_utils

print('Convert class vector to binary class matrix (for use with categorical_crossentropy)')
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)
print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
('Y_train shape:', (9595, 2))
('Y_test shape:', (1067, 2))


In [7]:
model = build_lstm(max_features, 64, nb_classes)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, None, 64)      640000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 64)            33024       embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 2)             130         lstm_1[0][0]                     
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 2)             0           dense_1[0][0]                    
Total params: 673,154
Trainable params: 673,154
Non-trainable params: 0
___________________

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [9]:
print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=5,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 9595 samples, validate on 1067 samples
Epoch 1/5
9595/9595 [==============================] - 23s - loss: 0.6469 - acc: 0.6090 - val_loss: 0.5461 - val_acc: 0.7329
Epoch 2/5
9595/9595 [==============================] - 23s - loss: 0.4638 - acc: 0.7842 - val_loss: 0.5053 - val_acc: 0.7488
Epoch 3/5
9595/9595 [==============================] - 22s - loss: 0.3517 - acc: 0.8425 - val_loss: 0.5172 - val_acc: 0.7601
Epoch 4/5
9595/9595 [==============================] - 23s - loss: 0.2702 - acc: 0.8931 - val_loss: 0.5912 - val_acc: 0.7619
Epoch 5/5
1067/1067 [==============================] - 1s     
('Test score:', 0.583477401632847)
('Test accuracy:', 0.75820056226841182)


In [10]:
# set parameters:
embedding_dims = 128 # size of embedding dims
nb_filter = 65      # number of filters 
filter_length = 5    # 1d convolution size
hidden_dims = 250    # size of hidden layers 


print('Build model...')
cnn_model = build_cnn(max_features,embedding_dims, maxlen, nb_filter, filter_length, hidden_dims, nb_classes )

Build model...


In [11]:
cnn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 60, 128)       1280000     embedding_input_2[0][0]          
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 56, 65)        41665       embedding_2[0][0]                
____________________________________________________________________________________________________
maxpooling1d_1 (MaxPooling1D)    (None, 1, 65)         0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 65)            0           maxpooling1d_1[0][0]             
___________________________________________________________________________________________

In [12]:
cnn_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
nb_epoch = 5

history = cnn_model.fit(X_train, Y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(X_test, Y_test))

Train on 9595 samples, validate on 1067 samples
Epoch 1/5
9595/9595 [==============================] - 2s - loss: 0.6148 - acc: 0.6416 - val_loss: 0.5213 - val_acc: 0.7526
Epoch 2/5
9595/9595 [==============================] - 2s - loss: 0.4088 - acc: 0.8132 - val_loss: 0.5013 - val_acc: 0.7676
Epoch 3/5
9595/9595 [==============================] - 2s - loss: 0.2728 - acc: 0.8865 - val_loss: 0.5303 - val_acc: 0.7657
Epoch 4/5
9595/9595 [==============================] - 2s - loss: 0.1809 - acc: 0.9279 - val_loss: 0.6321 - val_acc: 0.7338
Epoch 5/5
9595/9595 [==============================] - 2s - loss: 0.1297 - acc: 0.9483 - val_loss: 0.6753 - val_acc: 0.7460
